In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
data_url = 'https://qwer.gg/leagues/LCK/2021/summer'
driver = webdriver.Chrome("/Users/sjoon/Downloads/chromedriver")

In [4]:
driver.get(data_url)

In [5]:
lck = pd.DataFrame()

In [6]:
#채팅창 최소화
driver.find_element_by_class_name('Chat__header').click()

In [7]:
#더보기 끝까지
while True:
    try:
        driver.find_element_by_class_name('Tournament__records__more__button ').click()
        time.sleep(1)
    except Exception as e:
        pass
        break

In [8]:
match_url_list = []

In [9]:
#리스트에 매치 저장
matchlist = driver.find_elements_by_css_selector('a.MatchItem__link')

for match in matchlist:
    match_url = match.get_attribute('href')
    match_url_list.append(match_url)

In [10]:
len(match_url_list)

95

In [11]:
#매치에서 데이터 수집
for match_url in match_url_list:
    driver.get(match_url)
    
    #매치 세트별 정보 수집
    for match_set in driver.find_elements_by_css_selector('div.Tabs__label.Match__tabs__label'):
        match_set.click()
        time.sleep(2)
        
        #골드 그래프 선택
        driver.find_elements_by_css_selector('div.Tabs.MatchHistoryDetail__tabs div.Tabs__label.MatchHistoryDetail__tabs__label')[5].click()
    
        #골드그래프를 선택하기 위해 스크롤
        time.sleep(1)
        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2.4);")
    
        #골드격차 안뜨는 오류 먼저 한번 클릭
        driver.find_element_by_css_selector('path.highcharts-point').click()
        
        #블루팀 레드팀의 이름 확인
        blue_n = driver.find_elements_by_css_selector('span.MatchHistoryDetailResult__team')[0].text
        red_n =  driver.find_elements_by_css_selector('span.MatchHistoryDetailResult__team')[1].text
        blue_g = []
        red_g = []
        
        #골드 정보 받아오기
        points = driver.find_elements_by_css_selector('path.highcharts-point')
        
        for point in points:
    
            point.click()
            gold = driver.find_elements_by_tag_name('tspan')[1].text.replace("골드격차: ","")
            gold = gold.replace(",","")
            
            if driver.find_elements_by_tag_name('tspan')[1] == "격차없음":
                blue_g.append(0)
                red_g.append(0)
            if point.get_attribute('fill') == "#1f85fd":
                blue_g.append(gold)
                red_g.append('-' + gold)
            elif point.get_attribute('fill') == '#f60c50':
                blue_g.append('-' + gold)
                red_g.append(gold)
        total_gold = pd.DataFrame([blue_g, red_g], 
                                  index = ['blue_gold', 'red_gold'])
        
        total_gol = total_gold.rename(index={'blue_gold' : blue_n}, inplace=True)
        total_gol = total_gold.rename(index={'red_gold' : red_n}, inplace=True)
        
        
        lck = lck.append(total_gold)
        


In [13]:
lck

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
BRO,77,22,-120,-18,244,792,761,1167,1257,1355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LSB,-77,-22,120,18,-244,-792,-761,-1167,-1257,-1355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LSB,44,380,563,744,1059,726,498,1311,672,328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRO,-44,-380,-563,-744,-1059,-726,-498,-1311,-672,-328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,77,-228,23,720,700,868,682,918,814,1209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1,61,368,376,178,105,-9,349,81,451,1421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,121,536,346,466,208,783,681,1780,2136,2082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DK,-121,-536,-346,-466,-208,-783,-681,-1780,-2136,-2082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,-22,-416,-509,24,139,-115,145,179,164,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
lck.to_csv("lck_totalgold.csv")

In [15]:
lck_sort = lck.sort_index()

In [16]:
lck_sort

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
AF,20,38,188,543,596,-454,-68,-331,-291,-988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AF,-4,261,309,697,906,752,803,923,83,199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AF,-56,-167,1941,1476,2095,1794,1703,2075,1623,1426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AF,-20,-79,-299,318,290,530,343,182,436,580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AF,51,347,106,173,96,162,323,897,1176,1225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1,30,67,610,374,14,-48,-298,-213,-238,-369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,2,578,231,13,336,692,-725,-398,-701,-513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,45,120,185,592,563,1255,1139,204,945,1025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T1,-26,23,-46,401,412,402,1495,1705,1614,1389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
lck_sort.to_csv("lck_totalgold_sort.csv")